In [1]:
import matplotlib.pyplot as plt
import polars as pl

import numpy as np
import pandas as pd

import polars as pl
pl.enable_string_cache()
import seaborn as sns

%reload_ext autoreload
%autoreload 2

In [2]:
df_train = pl.read_parquet("../experiments/datasets/master_card_basic/train.parquet")
df_test = pl.read_parquet("../experiments/datasets/master_card_basic/test.parquet")

In [ ]:
X, y = train_dataset[features], train_dataset[config.target]

In [ ]:
from sklearn.linear_model import LogisticRegression


target = ["is_fraud"]
X_feautures = ["amount", "channel", "currency", "device"]
categorical_columns = ["channel", "currency", "device"]
X_feautures = ["amount", "user_id", "lat", "long"]
categorical_columns = [
    "user_id",
]

X_train, y_train = df_train[X_feautures], df_train[target]
X_train = X_train.with_columns(pl.col(c).to_physical() for c in categorical_columns)

X_test, y_test = df_test[X_feautures], df_test[target]
X_test = X_test.with_columns(pl.col(c).to_physical() for c in categorical_columns)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


X_train

transformer = make_column_transformer((StandardScaler(), ["amount"]), (OneHotEncoder(sparse_output=True), categorical_columns))
pipe = make_pipeline(transformer, LGBMClassifier(class_weight="balanced"))
pipe.fit(X_train, y_train.to_numpy().flatten())

[LightGBM] [Info] Number of positive: 35208, number of negative: 381534
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 26931
[LightGBM] [Info] Number of data points in the train set: 416742, number of used features: 13339
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['amount']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['user_id'])])),
                ('lgbmclassifier', LGBMClassifier(class_weight='balanced'))])

In [61]:
y_hat = pipe.predict(X_test)
y_hat

/home/quak/development/mastercard/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


array([0, 1, 1, ..., 0, 0, 1], shape=(83258,))

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score


{
    "accuracy": accuracy_score(y_test, y_hat),
    "auc": roc_auc_score(y_test, y_hat),
    "f1_score": f1_score(y_test, y_hat),
    "average_precision": average_precision_score(y_test, y_hat),
}


{'accuracy': 0.8084988829902232,
 'auc': np.float64(0.498023903344972),
 'f1_score': 0.09971767363071711,
 'average_precision': np.float64(0.08621080439461805)}

In [ ]:
y_hat = pipe.predict(X_train)


{
    "accuracy": accuracy_score(y_train, y_hat),
    "auc": roc_auc_score(y_train, y_hat),
    "f1_score": f1_score(y_train, y_hat),
    "average_precision": average_precision_score(y_train, y_hat),
}


/home/quak/development/mastercard/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


{'accuracy': 0.8205148509149546,
 'auc': np.float64(0.5450163104265328),
 'f1_score': 0.16735498090902007,
 'average_precision': np.float64(0.09582669115676182)}

In [3]:
import os
from mastercard.data_proc.src import compute_time_features, compute_user_time_statistics, interesting_features
from mastercard.models.mc_logistic.artifacts import Artifacts
import joblib

path = "../experiments/artifacts/00213eed2748424fbb5b8b32a86e2650/4f605141e4fa427888532c8cea0229e1"

artifacts = Artifacts(
    model=joblib.load(f"{path}/model.joblib"),
    features=joblib.load(f"{path}/features.joblib"),
    hyperparams=joblib.load(f"{path}/hyperparams.joblib"),
)

In [5]:


if artifacts.hyperparams["user_statistics"]:
    user_statistics, _ = compute_user_time_statistics(pl.concat([artifacts.train_dataset, df_test]))
    for stats in user_statistics.values():
        df_test = df_test.join_asof(stats, on="timestamp", by="user_id")

if artifacts.hyperparams["time_features"]:
    df_test, _ = compute_time_features(df_test)

df_test, _ = interesting_features(df_test)


df_test[artifacts.features]

amount,session_length_seconds,age_user,sum_of_monthly_installments_user,sum_of_monthly_expenses_user,risk_score_user,trust_score_merchant,number_of_alerts_last_6_months_merchant,avg_transaction_amount_merchant,account_age_months_merchant,has_fraud_history_merchant,same_country,integer_amount,channel,currency,device,payment_method,is_international,sex_user,education_user,primary_source_of_income_user,country_user,category_merchant,country_merchant
f64,i64,i64,f64,f64,f64,f64,i64,f64,i64,i64,bool,bool,cat,cat,cat,cat,i64,cat,cat,cat,cat,cat,cat
76.6,797,26,13.3,1469.69,0.615435,0.872707,3,245.06,70,0,false,false,"""in-store""","""EUR""","""iOS""","""credit_card""",1,"""Male""","""PhD""","""Employment""","""Denmark""","""grocery""","""Belgium"""
50.3,43,50,253.99,386.65,0.383892,0.639348,2,14.03,39,1,false,false,"""online""","""EUR""","""iOS""","""credit_card""",1,"""Female""","""PhD""","""Retirement""","""Germany""","""gaming""","""Poland"""
13.94,136,62,83.34,413.85,0.367631,0.889829,1,97.36,90,0,false,false,"""in-store""","""EUR""","""Android""","""debit_card""",1,"""Female""","""Bachelor""","""Business""","""Sweden""","""clothing""","""Poland"""
102.26,976,69,123.26,413.06,0.320273,0.879261,2,27.97,109,0,false,false,"""mobile""","""EUR""","""Web""","""debit_card""",1,"""Female""","""None""","""Business""","""France""","""gaming""","""Finland"""
87.8,969,41,384.33,2323.0,0.59351,0.856483,3,54.66,90,0,false,false,"""mobile""","""EUR""","""iOS""","""credit_card""",1,"""Other""","""PhD""","""Retirement""","""Finland""","""education""","""Sweden"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
25.85,62,36,13.36,293.94,0.623402,0.704613,3,38.33,36,0,false,false,"""in-store""","""EUR""","""Android""","""credit_card""",1,"""Female""","""Master""","""Employment""","""Finland""","""travel""","""Poland"""
13.04,992,42,124.44,1616.87,0.614074,0.751758,3,448.33,16,1,true,false,"""in-store""","""EUR""","""iOS""","""bank_transfer""",0,"""Female""","""PhD""","""Unemployment""","""Poland""","""restaurants""","""Poland"""
17.12,77,21,316.76,2362.34,0.468036,0.626142,2,14.07,20,0,false,false,"""in-store""","""EUR""","""Web""","""bank_transfer""",1,"""Male""","""PhD""","""Unemployment""","""Poland""","""electronics""","""Denmark"""


In [8]:
artifacts.model[0].transform(df_test[artifacts.features])

amount,session_length_seconds,age_user,sum_of_monthly_installments_user,sum_of_monthly_expenses_user,risk_score_user,trust_score_merchant,number_of_alerts_last_6_months_merchant,avg_transaction_amount_merchant,account_age_months_merchant,has_fraud_history_merchant,same_country,integer_amount,channel_in-store,channel_mobile,channel_online,currency_EUR,device_Android,device_Web,device_iOS,payment_method_bank_transfer,payment_method_credit_card,payment_method_debit_card,payment_method_mobile_payment,is_international,sex_user_Female,sex_user_Male,sex_user_Other,education_user_Bachelor,education_user_High School,education_user_Master,education_user_None,education_user_PhD,primary_source_of_income_user_Business,primary_source_of_income_user_Employment,primary_source_of_income_user_Retirement,primary_source_of_income_user_Savings,primary_source_of_income_user_Student Aid,primary_source_of_income_user_Unemployment,country_user_Austria,country_user_Belgium,country_user_Czech Republic,country_user_Denmark,country_user_Finland,country_user_France,country_user_Germany,country_user_Ireland,country_user_Italy,country_user_Netherlands,country_user_Poland,country_user_Portugal,country_user_Spain,country_user_Sweden,category_merchant_clothing,category_merchant_education,category_merchant_electronics,category_merchant_gaming,category_merchant_grocery,category_merchant_restaurants,category_merchant_travel,country_merchant_Austria,country_merchant_Belgium,country_merchant_Czech Republic,country_merchant_Denmark,country_merchant_Finland,country_merchant_France,country_merchant_Germany,country_merchant_Ireland,country_merchant_Italy,country_merchant_Netherlands,country_merchant_Poland,country_merchant_Portugal,country_merchant_Spain,country_merchant_Sweden
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
76.599998,797.0,26.0,13.3,1469.689941,0.615435,0.872707,3.0,245.059998,70.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50.299999,43.0,50.0,253.990005,386.649994,0.383892,0.639348,2.0,14.03,39.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13.94,136.0,62.0,83.339996,413.850006,0.367631,0.889829,1.0,97.360001,90.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
102.260002,976.0,69.0,123.260002,413.059998,0.320273,0.879261,2.0,27.969999,109.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87.800003,969.0,41.0,384.329987,2323.0,0.59351,0.856483,3.0,54.66,90.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
25.85,62.0,36.0,13.36,293.940002,0.623402,0.704613,3.0,38.330002,36.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.

In [9]:
df_test[artifacts.features]

amount,session_length_seconds,age_user,sum_of_monthly_installments_user,sum_of_monthly_expenses_user,risk_score_user,trust_score_merchant,number_of_alerts_last_6_months_merchant,avg_transaction_amount_merchant,account_age_months_merchant,has_fraud_history_merchant,same_country,integer_amount,channel,currency,device,payment_method,is_international,sex_user,education_user,primary_source_of_income_user,country_user,category_merchant,country_merchant
f64,i64,i64,f64,f64,f64,f64,i64,f64,i64,i64,bool,bool,cat,cat,cat,cat,i64,cat,cat,cat,cat,cat,cat
76.6,797,26,13.3,1469.69,0.615435,0.872707,3,245.06,70,0,false,false,"""in-store""","""EUR""","""iOS""","""credit_card""",1,"""Male""","""PhD""","""Employment""","""Denmark""","""grocery""","""Belgium"""
50.3,43,50,253.99,386.65,0.383892,0.639348,2,14.03,39,1,false,false,"""online""","""EUR""","""iOS""","""credit_card""",1,"""Female""","""PhD""","""Retirement""","""Germany""","""gaming""","""Poland"""
13.94,136,62,83.34,413.85,0.367631,0.889829,1,97.36,90,0,false,false,"""in-store""","""EUR""","""Android""","""debit_card""",1,"""Female""","""Bachelor""","""Business""","""Sweden""","""clothing""","""Poland"""
102.26,976,69,123.26,413.06,0.320273,0.879261,2,27.97,109,0,false,false,"""mobile""","""EUR""","""Web""","""debit_card""",1,"""Female""","""None""","""Business""","""France""","""gaming""","""Finland"""
87.8,969,41,384.33,2323.0,0.59351,0.856483,3,54.66,90,0,false,false,"""mobile""","""EUR""","""iOS""","""credit_card""",1,"""Other""","""PhD""","""Retirement""","""Finland""","""education""","""Sweden"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
25.85,62,36,13.36,293.94,0.623402,0.704613,3,38.33,36,0,false,false,"""in-store""","""EUR""","""Android""","""credit_card""",1,"""Female""","""Master""","""Employment""","""Finland""","""travel""","""Poland"""
13.04,992,42,124.44,1616.87,0.614074,0.751758,3,448.33,16,1,true,false,"""in-store""","""EUR""","""iOS""","""bank_transfer""",0,"""Female""","""PhD""","""Unemployment""","""Poland""","""restaurants""","""Poland"""
17.12,77,21,316.76,2362.34,0.468036,0.626142,2,14.07,20,0,false,false,"""in-store""","""EUR""","""Web""","""bank_transfer""",1,"""Male""","""PhD""","""Unemployment""","""Poland""","""electronics""","""Denmark"""
